In [ ]:
# from google.colab import files
# uploaded = files.upload()

!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/TS_2.직접촬영_01.금속캔_001.철캔_2.zip' -d '/content/data'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !apt-get install p7zip-full

# !pip install py7zr

# import py7zr

# with py7zr.SevenZipFile('/content/drive/MyDrive/KCC3기_2차_프로젝트/TS_2.직접촬영_01.금속캔_001.철캔_2.7z', mode='r') as z:
#     z.extractall('/content/data')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.1 MB/s eta 0:00:00


checkdir:  cannot create extraction directory: /content/data/label
           No such file or directory


In [48]:
# zip 파일 압축 해제할 폴더(디렉터리) 생성 및 압축 해제

import os

base_dir = os.getcwd()

raw_path = base_dir + '/data/raw/'
label_path = base_dir + '/data/label/'
label_txt_path = base_dir + '/data/label_txt/'

os.makedirs(raw_path, exist_ok=True)
os.makedirs(label_path, exist_ok=True)

!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/rawtest.zip' -d '/content/data/raw/'
!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/labelingtest.zip' -d '/content/data/label/'

In [49]:
# 파일명 읽어들이기

import os

raw_list = os.listdir(raw_path)
label_list = os.listdir(label_path)

raw_list.sort()
label_list.sort()

print(raw_list[:5])
print(label_list[:5])


['561190@0_07001_220822_P1_T3__0147.jpg', '561192@0_07001_220822_P1_T3__0422.jpg', '561194@0_07001_220822_P1_T3__0589.jpg', '561195@0_07001_220822_P1_T3__0600.jpg', '561196@0_07001_220822_P1_T3__0674.jpg']
['561190@0_07001_220822_P1_T3__0147.json', '561192@0_07001_220822_P1_T3__0422.json', '561194@0_07001_220822_P1_T3__0589.json', '561195@0_07001_220822_P1_T3__0600.json', '561196@0_07001_220822_P1_T3__0674.json']


In [56]:
# json 데이터를 YOLO 데이터 포맷으로 변경하기

import json
import os
import pandas as pd
import numpy as np

# for label in label_list:
file_json = open(label_path + label_list[0])
f_json = json.load(file_json)


img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

print(img_width)
print(img_height)

obj_id = f_json['ANNOTATION_INFO'][0]['ID']
obj_points = np.array(f_json['ANNOTATION_INFO'][0]['POINTS'][0])
print(obj_id)
print(obj_points)

yolo_id = obj_id
yolo_points = np.array([obj_points[0]+obj_points[2]/2, obj_points[1]+obj_points[3]/2,
                             obj_points[2], obj_points[3]])

yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])

print(yolo_points)
print(yolo_points_norm)

f_txt = open(label_txt_path + label_list[0].replace('.json', '.txt'), 'w')
yolo_points_norm = list(yolo_points_norm)
f_txt.write(str(yolo_id) + ' ')
f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
f_txt.write('\n')
f_txt.close()


1920
1080
2
[169.48845426 106.4481388  440.44921136 342.76542587]
[389.71305994 277.83085174 440.44921136 342.76542587]
[0.20297555 0.25725079 0.22940063 0.31737539]


In [59]:
# json 데이터를 YOLO 데이터 포맷으로 변경하기

import json
import os
import pandas as pd
import numpy as np

for label in label_list:
  file_json = open(label_path + label)
  f_json = json.load(file_json)

  img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
  img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

  # print(img_width)
  # print(img_height)

  f_txt = open(label_txt_path + label.replace('.json', '.txt'), 'w')

  for anno in f_json['ANNOTATION_INFO']:
    obj_id = anno['ID']
    obj_points = np.array(anno['POINTS'][0])
    # print(obj_id)
    # print(obj_points)
    if obj_points.size == 4:
      yolo_id = obj_id
      yolo_points = np.array([obj_points[0]+obj_points[2]/2, obj_points[1]+obj_points[3]/2,
                              obj_points[2], obj_points[3]])

      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])

      # print(yolo_points)
      # print(yolo_points_norm)

      yolo_points_norm = list(yolo_points_norm)
      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    else:
      print(label)
  f_txt.close()



561324@0_07001_220822_P1_T3__0346.json
561324@0_07001_220822_P1_T3__0346.json
561324@0_07001_220822_P1_T3__0346.json
561324@0_07001_220822_P1_T3__0346.json
561324@0_07001_220822_P1_T3__0346.json
